In [1]:
import pandas as pd

In [2]:
dfP1 = pd.read_csv("/Users/reginaldocunha/Documents/Portfólio de Projetos Data Science/Projeto Produção Biodiesel/biodiesel-producao.csv")

In [3]:
dfP1

,Mês/Ano,Região,Produção de Biodiesel
0,06/2023,NORDESTE,49975
1,06/2023,NORTE / CENTRO OESTE,300341
2,06/2023,SUDESTE,29533
3,06/2023,SUL,274958
4,05/2023,NORDESTE,51346
...,...,...,...
307,02/2017,SUL,102263
308,01/2017,NORDESTE,20107
309,01/2017,NORTE / CENTRO OESTE,103414
310,01/2017,SUDESTE,24076


In [4]:
dfP1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 312 entries, 0 to 311
Data columns (total 3 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Mês/Ano                312 non-null    object
 1   Região                 312 non-null    object
 2   Produção de Biodiesel  312 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 7.4+ KB


In [6]:
dfP1["Mês/Ano"] = pd.to_datetime(dfP1["Mês/Ano"])

/var/folders/ps/1zsgwz6x7wq512m3wfzp15yr0000gn/T/ipykernel_11905/836807187.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dfP1["Mês/Ano"] = pd.to_datetime(dfP1["Mês/Ano"])


In [7]:
dfP1

,Mês/Ano,Região,Produção de Biodiesel
0,2023-06-01,NORDESTE,49975
1,2023-06-01,NORTE / CENTRO OESTE,300341
2,2023-06-01,SUDESTE,29533
3,2023-06-01,SUL,274958
4,2023-05-01,NORDESTE,51346
...,...,...,...
307,2017-02-01,SUL,102263
308,2017-01-01,NORDESTE,20107
309,2017-01-01,NORTE / CENTRO OESTE,103414
310,2017-01-01,SUDESTE,24076


In [ ]:
#Gerar plot manda por whatsApp
#